In [16]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

# Function to calculate sample size
def calculate_sample_size(proportion, z, margin_of_error):
    return np.ceil((z**2 * proportion * (1 - proportion)) / (margin_of_error**2))

# Function to update the plot and display the sample size
def update_plot(proportion=0.5, confidence_level=95, margin_of_error=0.05, total_population=1000):
    z_scores = {90: 1.645, 95: 1.96, 99: 2.576}
    z = z_scores[confidence_level]
    sample_size = calculate_sample_size(proportion, z, margin_of_error)
    sample_size = min(sample_size, total_population)  # Adjust for finite population
    
    # Generate population data
    x = np.linspace(0.01, 0.99, 100)
    y_sample = [calculate_sample_size(px, z, margin_of_error) for px in x]
    y_sample = [min(n, total_population) for n in y_sample]  # Adjust for total population

    # Plot population and sample size
    plt.figure(figsize=(10, 6))
    plt.plot(x, y_sample, label='Sample Size vs. Proportion', color='blue')
    plt.axvline(proportion, color='red', linestyle='--', label=f'Selected Proportion: {proportion}')
    plt.axhline(sample_size, color='green', linestyle='--', label=f'Calculated Sample Size: {sample_size}')
    plt.title(f'Sample Size and Total Population\nConfidence Level: {confidence_level}%, Margin of Error: {margin_of_error}')
    plt.xlabel('Proportion (p)')
    plt.ylabel('Sample Size (n)')
    plt.grid()
    plt.legend(loc='upper right')

    # Add total population for context
    plt.fill_between(x, y_sample, total_population, color='gray', alpha=0.2, label='Remaining Population')
    plt.legend()
    plt.show()

    # Update the output widget with the sample size using HTML for larger font
    sample_size_output.clear_output()
    with sample_size_output:
        display(HTML(f"<h2 style='font-size: 36px;'>Sample Size Needed: {int(sample_size)}</h2>"))

# Widgets for interactivity
proportion_slider = widgets.FloatSlider(value=0.5, min=0.01, max=0.99, step=0.01, description='Proportion (p):')
confidence_level_slider = widgets.IntSlider(value=95, min=90, max=99, step=1, description='Confidence Level:')
margin_of_error_slider = widgets.FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description='Margin of Error:')
total_population_slider = widgets.IntSlider(value=1000, min=100, max=10000, step=100, description='Total Population:')

# Create the output widget for sample size display
sample_size_output = widgets.Output()

# Interactive widget display
interactive_plot = widgets.interactive(
    update_plot,
    proportion=proportion_slider,
    confidence_level=confidence_level_slider,
    margin_of_error=margin_of_error_slider,
    total_population=total_population_slider
)

# Display the widgets and the output window
display(widgets.VBox([interactive_plot, sample_size_output]))


In [17]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import ipywidgets as widgets
from IPython.display import display

# Function to create the combined visualization
def combined_visualization(mean1=0, mean2=1, std1=1, std2=1, sample_size=30, alpha=0.05):
    # Generate random samples
    np.random.seed(2)
    group1 = np.random.normal(mean1, std1, sample_size)
    group2 = np.random.normal(mean2, std2, sample_size)

    # Calculate effect size (Cohen's d)
    pooled_std = np.sqrt(((std1**2) + (std2**2)) / 2)
    cohen_d = (mean2 - mean1) / pooled_std

    # Perform t-test
    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)
    df = (sample_size - 1) * 2  # Degrees of freedom approximation

    # Create the figure
    fig, axs = plt.subplots(2, 1, figsize=(12, 12), gridspec_kw={'height_ratios': [2, 1]})
    
    # Plot 1: Group Distributions and Effect Size
    bins = np.linspace(
        min(group1.min(), group2.min()), 
        max(group1.max(), group2.max()), 
        20
    )
    axs[0].hist(group1, bins=bins, alpha=0.7, label=f'Group 1 (Mean={mean1}, SD={std1})')
    axs[0].hist(group2, bins=bins, alpha=0.7, label=f'Group 2 (Mean={mean2}, SD={std2})')
    axs[0].axvline(mean1, color='blue', linestyle='--', label='Group 1 Mean')
    axs[0].axvline(mean2, color='orange', linestyle='--', label='Group 2 Mean')
    axs[0].text(
        0.05, 0.95, f"Cohen's d = {cohen_d:.2f}", 
        transform=axs[0].transAxes, fontsize=12, verticalalignment='top'
    )
    axs[0].text(
        0.05, 0.85, f"P-value = {p_value:.4f} {'(Significant)' if p_value < alpha else '(Not Significant)'}", 
        transform=axs[0].transAxes, fontsize=12, verticalalignment='top'
    )
    axs[0].set_title(f'Group Distributions and Effect Size\nα = {alpha} | P-value = {p_value:.4f}')
    axs[0].set_xlabel('Value')
    axs[0].set_ylabel('Frequency')
    axs[0].legend()
    axs[0].grid()

    # Plot 2: P-value on the T-distribution
    t_values = np.linspace(-4, 4, 500)
    t_dist = stats.t.pdf(t_values, df)
    critical_value = stats.t.ppf(1 - alpha / 2, df)

    axs[1].plot(t_values, t_dist, label='T-distribution', color='blue')
    axs[1].fill_between(t_values, 0, t_dist, where=(t_values <= -critical_value) | (t_values >= critical_value), color='red', alpha=0.3, label='Critical Region (α)')
    axs[1].axvline(t_stat, color='green', linestyle='--', label=f'T-statistic = {t_stat:.2f}')
    axs[1].text(
        t_stat, 0.02, f"P-value = {p_value:.4f}", color='green', fontsize=12, ha='center'
    )
    axs[1].set_title('T-distribution and P-value')
    axs[1].set_xlabel('T-statistic')
    axs[1].set_ylabel('Density')
    axs[1].legend()
    axs[1].grid()

    plt.tight_layout()
    plt.show()

# Widgets
mean1_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Mean 1:')
mean2_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Mean 2:')
std1_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='SD 1:')
std2_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='SD 2:')
sample_size_slider = widgets.IntSlider(value=30, min=5, max=100, step=1, description='Sample Size:')
alpha_slider = widgets.FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description='Alpha Level (α):')

# Interactive display
interactive_plot = widgets.interactive(
    combined_visualization,
    mean1=mean1_slider,
    mean2=mean2_slider,
    std1=std1_slider,
    std2=std2_slider,
    sample_size=sample_size_slider,
    alpha=alpha_slider
)

display(interactive_plot)


interactive(children=(FloatSlider(value=0.0, description='Mean 1:', max=5.0, min=-5.0), FloatSlider(value=1.0,…